# Automated ML

Install and import dependencies

In [1]:
!pip install kaggle

If the import of data_prep fails, see the README on how to download and copy the kaggle.json

In [3]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import data_prep

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ws = Workspace.from_config()
experiment_name = 'automl_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep.get_dataset(ws)

In [10]:
dataset = data_prep_malware.create_dataset(ws)

WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/4d18c7ab-3cf5-4b6a-844e-e9d3a4086127/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## Find or Create Compute Cluster

In [7]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
automl_settings =  {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "featurization": 'auto'
}

automl_config = AutoMLConfig(
                    task="classification",
                    training_data=dataset,
                    label_column_name="legitimate",
                    compute_target=cpu_cluster,
                    **automl_settings)

In [9]:
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_automl_run, best_automl_model = automl_run.get_output()
best_run_metrics = best_automl_run.get_metrics()

print('Best Run Id: ', best_automl_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(best_automl_model._final_estimator)
print(best_automl_run.get_tags())

Best Run Id:  AutoML_4f993bc8-389e-4fc5-a624-546a6dece85c_38

 Accuracy: 0.9999782684283728
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('1',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
         

In [31]:
model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [59]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "AzureML-Minimal").clone("my_env")

for pip_package in ["scikit-learn", "numpy"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score_automl.py',
                                    environment=env)

In [61]:
rest_service = Model.deploy(ws, "best-model-service", [model], inference_config=inference_config)

rest_service.wait_for_deployment(show_output=True)
rest_service.update(enable_app_insights=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cfeb3213-b2f9-4485-aef9-b259b9e8a824
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: best-model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image cc8f942d63e94020bbc9d2b49d596d1b.azurecr.io/azureml/azureml_ab7c7ad2e987034a58e11c3005c6abd9 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cfeb3213-b2f9-4485-aef9-b259b9e8a824
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: best-model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image cc8f942d63e94020bbc9d2b49d596d1b.azurecr.io/azureml/azureml_ab7c7ad2e987034a58e11c3005c6abd9 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring fil

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
import json

data = {"data": 
          [ 
            { 
              "Name": "memtest.exe", 
              "md5": "631ea355665f28d4707448e442fbf5b8", 
              "Machine": 332, 
              "SizeOfOptionalHeader": 224, 
              "Characteristics": 258, 
              "MajorLinkerVersion": 9, 
              "MinorLinkerVersion": 0, 
              "SizeOfCode": 361984, 
              "SizeOfInitializedData": 115712, 
              "SizeOfUninitializedData": 0, 
              "AddressOfEntryPoint": 6135, 
              "BaseOfCode": 4096, 
              "BaseOfData": 372736, 
              "ImageBase": 4194304, 
              "SectionAlignment": 4096, 
              "FileAlignment": 512, 
              "MajorOperatingSystemVersion": 0, 
              "MinorOperatingSystemVersion": 0, 
              "MajorImageVersion": 0, 
              "MinorImageVersion": 0, 
              "MajorSubsystemVersion": 1, 
              "MinorSubsystemVersion": 0, 
              "SizeOfImage": 1036288, 
              "SizeOfHeaders": 1024, 
              "CheckSum": 485887, 
              "Subsystem": 16, 
              "DllCharacteristics": 1024, 
              "SizeOfStackReserve": 1048576, 
              "SizeOfStackCommit": 4096, 
              "SizeOfHeapReserve": 1048576, 
              "SizeOfHeapCommit": 4096, 
              "LoaderFlags": 0, 
              "NumberOfRvaAndSizes": 16, 
              "SectionsNb": 8, 
              "SectionsMeanEntropy": 5.7668065537, 
              "SectionsMinEntropy": 3.6074295755, 
              "SectionsMaxEntropy": 7.2210507289, 
              "SectionsMeanRawsize": 59712, 
              "SectionsMinRawsize": 1024, 
              "SectionMaxRawsize": 325120, 
              "SectionsMeanVirtualsize": 126875.875, 
              "SectionsMinVirtualsize": 896, 
              "SectionMaxVirtualsize": 551848, 
              "ImportsNbDLL": 0, 
              "ImportsNb": 0, 
              "ImportsNbOrdinal": 0, 
              "ExportNb": 0, 
              "ResourcesNb": 4, 
              "ResourcesMeanEntropy": 3.262822711, 
              "ResourcesMinEntropy": 2.5688438236, 
              "ResourcesMaxEntropy": 3.5379393642, 
              "ResourcesMeanSize": 8797, 
              "ResourcesMinSize": 216, 
              "ResourcesMaxSize": 18032, 
              "LoadConfigurationSize": 0, 
              "VersionInformationSize": 16 
            } 
          ]
        }

# Convert to JSON string
input_data = json.dumps(data)

In [ ]:
import requests


response = requests.post(rest_service, 
                         headers=auth_header, 
                         json=input_data
                        )

print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [62]:
logs = rest_service.get_logs()

for line in logs.split('\n'):
    print(line)

2020-11-17T16:45:09.9174302Z stderr F /bin/bash: /azureml-envs/azureml_75f566fe4b3478240add6787b2264ded/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-11-17T16:45:09.9254401Z stderr F /bin/bash: /azureml-envs/azureml_75f566fe4b3478240add6787b2264ded/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-11-17T16:45:09.9264246Z stderr F /bin/bash: /azureml-envs/azureml_75f566fe4b3478240add6787b2264ded/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-11-17T16:45:09.9374282Z stdout F 2020-11-17T16:45:09,069629800+00:00 - gunicorn/run 
2020-11-17T16:45:09.9384289Z stderr F /bin/bash: /azureml-envs/azureml_75f566fe4b3478240add6787b2264ded/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-11-17T16:45:10.0178721Z stdout F 2020-11-17T16:45:09,149049200+00:00 - iot-server/run 
2020-11-17T16:45:10.0564269Z stdout F 2020-11-17T16:45:09,181474200+00:00 - nginx/run 
2020-11-17T